In [1]:
# Load the cleaned dataset
import pandas as pd

In [2]:
# Load the dataset
df = pd.read_csv("release_cleaned_data.csv")

In [3]:
print("Shape of dataset:", df.shape)
print("\nColumn Names:\n", df.columns.tolist())

Shape of dataset: (737516, 19)

Column Names:
 ['reporting_year', 'npri_id', 'company_name', 'naics_code', 'naics_title', 'province', 'city', 'latitude', 'longitude', 'cas_number', 'substance_name', 'units', 'release_air_fugitive', 'release_air_other_non_point', 'release_air_road_dust', 'release_air_spills', 'release_air_stack_point', 'release_air_storage_handling', 'release_total']


In [4]:
# Show data types and check for nulls
print("Data Types:\n", df.dtypes)
print("\nMissing values:\n", df.isnull().sum())

Data Types:
 reporting_year                   object
npri_id                           int64
company_name                     object
naics_code                        int64
naics_title                      object
province                         object
city                             object
latitude                        float64
longitude                       float64
cas_number                       object
substance_name                   object
units                            object
release_air_fugitive            float64
release_air_other_non_point     float64
release_air_road_dust           float64
release_air_spills              float64
release_air_stack_point         float64
release_air_storage_handling    float64
release_total                   float64
dtype: object

Missing values:
 reporting_year                       0
npri_id                              0
company_name                         0
naics_code                           0
naics_title                          0


In [5]:
# Preview the top rows
df.head()

,reporting_year,npri_id,company_name,naics_code,naics_title,province,city,latitude,longitude,cas_number,substance_name,units,release_air_fugitive,release_air_other_non_point,release_air_road_dust,release_air_spills,release_air_stack_point,release_air_storage_handling,release_total
0,2000-01-01,1,Alberta-Pacific Forest Industries Inc.,322112,Chemical pulp mills,AB,County of Athabasca,54.923116,-112.861867,10049-04-4,Chlorine dioxide,tonnes,0.015,2.045,370.095,0.471211,5.200000,0.196,0.003
1,2000-01-01,1,Alberta-Pacific Forest Industries Inc.,322112,Chemical pulp mills,AB,County of Athabasca,54.923116,-112.861867,67-56-1,Methanol,tonnes,0.015,2.045,370.095,0.471211,98.056101,0.196,0.003
2,2000-01-01,1,Alberta-Pacific Forest Industries Inc.,322112,Chemical pulp mills,AB,County of Athabasca,54.923116,-112.861867,67-66-3,Chloroform,tonnes,0.015,2.045,370.095,0.471211,60.335000,0.196,0.003
3,2000-01-01,1,Alberta-Pacific Forest Industries Inc.,322112,Chemical pulp mills,AB,County of Athabasca,54.923116,-112.861867,75-07-0,Acetaldehyde,tonnes,0.015,2.045,370.095,0.471211,7.670000,0.196,0.003
4,2000-01-01,1,Alberta-Pacific Forest Industries Inc.,322112,Chemical pulp mills,AB,County of Athabasca,54.923116,-112.861867,7647-01-0,Hydrochloric acid,tonnes,0.015,2.045,370.095,0.471211,0.665000,0.196,0.003


In [6]:
# Create a copy of df for safety
df_filled = df.copy()

# Step 1: Create precise lat/lon key
df_filled['latlon_key'] = df_filled['latitude'].astype(str) + "_" + df_filled['longitude'].astype(str)

# Map from exact coordinates
exact_map = df_filled[df_filled['city'].notnull()].groupby('latlon_key')['city'].agg(lambda x: x.mode()[0])
df_filled.loc[df_filled['city'].isnull(), 'city'] = df_filled[df_filled['city'].isnull()]['latlon_key'].map(exact_map)

# Step 2–4: Round lat/lon and match with coarser precision
for r in [3, 2, 1]:
    key_name = f'latlon_r{r}'
    df_filled[key_name] = df_filled['latitude'].round(r).astype(str) + "_" + df_filled['longitude'].round(r).astype(str)

    map_r = df_filled[df_filled['city'].notnull()].groupby(key_name)['city'].agg(lambda x: x.mode()[0])
    df_filled.loc[df_filled['city'].isnull(), 'city'] = df_filled[df_filled['city'].isnull()][key_name].map(map_r)

# Step 5: Fill remaining with placeholder
df_filled['city'] = df_filled['city'].fillna("Unknown City")

# Cleanup helper columns
df_filled.drop(columns=['latlon_key', 'latlon_r3', 'latlon_r2', 'latlon_r1'], inplace=True)

# Final check
print(" All city values filled? Missing city count:", df_filled['city'].isnull().sum())
df = df_filled.copy()  # Replace original with fully filled one

 All city values filled? Missing city count: 0


In [7]:
# Count how many rows have "Unknown City"
unknown_count = (df['city'] == "Unknown City").sum()
print(" Rows with 'Unknown City':", unknown_count)

# Optional: preview a few
df[df['city'] == "Unknown City"].head(10)

 Rows with 'Unknown City': 58986


,reporting_year,npri_id,company_name,naics_code,naics_title,province,city,latitude,longitude,cas_number,substance_name,units,release_air_fugitive,release_air_other_non_point,release_air_road_dust,release_air_spills,release_air_stack_point,release_air_storage_handling,release_total
1296,2000-01-01,4136,CANADIAN NATURAL RESOURCES LTD.,211114,Non-conventional oil extraction,AB,Unknown City,54.7729,-110.5790,108-88-3,Toluene,tonnes,0.270213,46.131561,370.095,0.036680,0.0412,0.026000,0.000736
1297,2000-01-01,4136,CANADIAN NATURAL RESOURCES LTD.,211114,Non-conventional oil extraction,AB,Unknown City,54.7729,-110.5790,1330-20-7,Xylene (all isomers),tonnes,0.540426,46.131561,370.095,0.038515,0.0314,0.021000,0.000981
1298,2000-01-01,4136,CANADIAN NATURAL RESOURCES LTD.,211114,Non-conventional oil extraction,AB,Unknown City,54.7729,-110.5790,71-43-2,Benzene,tonnes,0.810638,46.131561,370.095,0.040351,0.0216,0.015000,0.001226
1299,2000-01-01,4136,CANADIAN NATURAL RESOURCES LTD.,211114,Non-conventional oil extraction,AB,Unknown City,54.7729,-110.5790,7647-01-0,Hydrochloric acid,tonnes,1.080851,46.131561,370.095,0.042186,0.0118,0.121538,0.001472
1503,2000-01-01,5239,AEC Oil & Gas,211113,Conventional oil and gas extraction,AB,Unknown City,55.4891,-119.7508,75-15-0,Carbon disulphide,tonnes,5.704000,46.131561,370.095,0.005077,0.0360,3.711667,0.008451
1504,2000-01-01,5239,AEC Oil & Gas,211113,Conventional oil and gas extraction,AB,Unknown City,55.4891,-119.7508,7783-06-04,Hydrogen sulphide,tonnes,7.438000,46.131561,370.095,0.005055,0.1030,4.260000,0.008525
1505,2000-01-01,5239,AEC Oil & Gas,211113,Conventional oil and gas extraction,AB,Unknown City,55.4891,-119.7508,7783-06-04,Hydrogen sulphide,tonnes,6.651875,46.131561,370.095,0.005033,0.1700,4.062476,0.008598
1650,2000-01-01,5395,CANADIAN NATURAL RESOURCES LTD.,211114,Non-conventional oil extraction,AB,Unknown City,55.9935,-113.4805,107-21-1,Ethylene glycol,tonnes,1.386000,46.131561,370.095,0.001846,0.4160,0.149474,0.232750
1651,2000-01-01,5395,CANADIAN NATURAL RESOURCES LTD.,211114,Non-conventional oil extraction,AB,Unknown City,55.9935,-113.4805,108-88-3,Toluene,tonnes,1.188000,46.131561,370.095,0.001824,0.3480,0.004000,0.207000
1652,2000-01-01,5395,CANADIAN NATURAL RESOURCES LTD.,211114,Non-conventional oil extraction,AB,Unknown City,55.9935,-113.4805,110-54-3,n-Hexane,tonnes,0.990000,46.131561,370.095,0.001802,0.2800,0.155000,0.181250


In [8]:
# Only apply to "Unknown City" rows
unknown_mask = df['city'] == "Unknown City"

# Step: Group by company + province + industry
company_prov_naics_map = (
    df[~unknown_mask]  # only where city is known
    .groupby(['company_name', 'province', 'naics_code'])['city']
    .agg(lambda x: x.mode()[0] if not x.mode().empty else None)
)

# Apply the map to rows with "Unknown City"
def smart_fill_city(row):
    if row['city'] != "Unknown City":
        return row['city']
    key = (row['company_name'], row['province'], row['naics_code'])
    return company_prov_naics_map.get(key, "Unknown City")  # fallback again if no match

# Fill city with new guesses
df['city'] = df.apply(smart_fill_city, axis=1)

# Final count check
print(" Remaining 'Unknown City' rows after smart fallback:", (df['city'] == "Unknown City").sum())

 Remaining 'Unknown City' rows after smart fallback: 2372


In [9]:
# List of CACs to include
cac_list = [
    "Sulphur dioxide (SOX)",
    "Nitrogen oxides (NOX)",
    "Volatile organic compounds (VOCs)",
    "Particulate Matter <= 10 Microns (PM10)",
    "Particulate Matter <= 2.5 Microns (PM2.5)",
    "Carbon monoxide (CO)"
]

# Filter the dataset
df_cac = df[df['substance_name'].isin(cac_list)].copy()

# Check counts per pollutant
print(" Rows per pollutant:")
print(df_cac['substance_name'].value_counts())

# Quick preview
df_cac[['reporting_year', 'province', 'substance_name', 'release_total']].head()


 Rows per pollutant:
Series([], Name: count, dtype: int64)


,reporting_year,province,substance_name,release_total


In [10]:
# List all unique substance names
print(df['substance_name'].unique())

['Chlorine dioxide' 'Methanol' 'Chloroform' 'Acetaldehyde'
 'Hydrochloric acid' 'Phosphoric acid' 'Sulphuric acid' 'Chlorine'
 'Manganese (and its compounds)' 'Zinc (and its compounds)'
 'Ammonia (total)' 'Nitrate ion in solution at pH >= 6.0'
 'Hydrogen sulphide' 'Phenol (and its salts)'
 'Diethanolamine (and its salts)' 'Formaldehyde' 'Ethylene glycol'
 'Nonylphenol polyethylene glycol ether' 'Naphthalene'
 '1,2,4-Trimethylbenzene' 'Toluene' 'n-Hexane' 'Cyclohexane'
 'Xylene (all isomers)' 'Benzene' 'Vinyl acetate' 'Propylene' 'Ethylene'
 'Mercury (and its compounds)' 'Toluenediisocyanate (mixed isomers)'
 'Dichloromethane' 'Dioxins and furans - total' 'Hexachlorobenzene'
 'Hydrogen fluoride' 'Chromium (and its compounds)'
 'Copper (and its compounds)' 'Lead (and its compounds)'
 'Nickel (and its compounds)' 'Styrene' '1,3-Butadiene'
 '1,2-Dichloroethane' '2,4-Dichlorophenol (and its salts)' '1,4-Dioxane'
 'Tetrachloroethylene' 'HCFC-123 (all isomers)' 'Carbon tetrachloride'
 'Formic

In [11]:
# Normalize substance names to lowercase for easy matching
df['substance_name_lower'] = df['substance_name'].str.lower()

# Build a mask for CACs using keyword matching
mask_cac = (
    df['substance_name_lower'].str.contains("sulphur dioxide") |
    df['substance_name_lower'].str.contains("nitrogen oxides") |
    df['substance_name_lower'].str.contains("volatile organic") |
    df['substance_name_lower'].str.contains("carbon monoxide") |
    df['substance_name_lower'].str.contains("pm10") |
    df['substance_name_lower'].str.contains("pm2.5")
)

# Filter using the mask
df_cac = df[mask_cac].copy()

# Drop helper column
df_cac.drop(columns='substance_name_lower', inplace=True)

# Check results
print("Matched CAC pollutants:")
print(df_cac['substance_name'].value_counts())

# Preview
df_cac[['reporting_year', 'province', 'substance_name', 'release_total']].head()

Matched CAC pollutants:
substance_name
PM2.5 - Particulate Matter <= 2.5 Micrometers      88440
PM10 - Particulate Matter <= 10 Micrometers        83139
Nitrogen oxides (expressed as nitrogen dioxide)    73198
Carbon monoxide                                    60232
Volatile Organic Compounds (VOCs)                  58988
Sulphur dioxide                                    26828
Volatile Organic Compounds (Total)                  6255
Name: count, dtype: int64


,reporting_year,province,substance_name,release_total
20037,2002-01-01,AB,Nitrogen oxides (expressed as nitrogen dioxide),0.002063
20040,2002-01-01,AB,Carbon monoxide,0.002156
20059,2002-01-01,AB,PM10 - Particulate Matter <= 10 Micrometers,0.002750
20060,2002-01-01,AB,PM2.5 - Particulate Matter <= 2.5 Micrometers,0.002781
20061,2002-01-01,AB,Volatile Organic Compounds (VOCs),0.002812


In [12]:
# Convert reporting_year to datetime, then extract year
df_cac['year'] = pd.to_datetime(df_cac['reporting_year']).dt.year

# Group and sum total releases
df_yearly = (
    df_cac.groupby(['year', 'substance_name'])['release_total']
    .sum()
    .reset_index()
)

# Pivot so each pollutant is its own column
df_pivot = df_yearly.pivot(index='year', columns='substance_name', values='release_total')

# Sort years
df_pivot.sort_index(inplace=True)

# Display final time series DataFrame
print("Shape of time series data:", df_pivot.shape)
df_pivot.head()

Shape of time series data: (21, 7)


substance_name,Carbon monoxide,Nitrogen oxides (expressed as nitrogen dioxide),PM10 - Particulate Matter <= 10 Micrometers,PM2.5 - Particulate Matter <= 2.5 Micrometers,Sulphur dioxide,Volatile Organic Compounds (Total),Volatile Organic Compounds (VOCs)
year,,,,,,,
2002,147.246847,159.059388,253.973166,243.146041,76.784369,NaN,214.741881
2003,216.480294,285.124659,346.596839,355.166858,99.091543,NaN,240.128918
2004,419.821364,585.752857,495.497311,564.858657,143.403817,NaN,344.756318
2005,359.751796,509.819696,462.171837,517.097923,114.424408,NaN,320.663002
2006,350.710360,522.170522,521.081071,592.825515,108.456482,NaN,323.519618


In [13]:
# Drop redundant or incomplete column if needed
df_pivot.drop(columns=['Volatile Organic Compounds (Total)'], inplace=True)

# Rename columns for easier model handling (optional)
df_pivot.columns = [
    'Carbon_monoxide',
    'Nitrogen_oxides',
    'PM10',
    'PM2.5',
    'Sulphur_dioxide',
    'VOCs'
]

# Final time series preview
print("Final pollutants for modeling:", df_pivot.columns.tolist())
df_pivot.head()

Final pollutants for modeling: ['Carbon_monoxide', 'Nitrogen_oxides', 'PM10', 'PM2.5', 'Sulphur_dioxide', 'VOCs']


,Carbon_monoxide,Nitrogen_oxides,PM10,PM2.5,Sulphur_dioxide,VOCs
year,,,,,,
2002,147.246847,159.059388,253.973166,243.146041,76.784369,214.741881
2003,216.480294,285.124659,346.596839,355.166858,99.091543,240.128918
2004,419.821364,585.752857,495.497311,564.858657,143.403817,344.756318
2005,359.751796,509.819696,462.171837,517.097923,114.424408,320.663002
2006,350.710360,522.170522,521.081071,592.825515,108.456482,323.519618


In [14]:
# Step 1: Create lag-1 features for each pollutant
df_lagged = df_pivot.copy()

# Create a lag-1 column for each pollutant
for col in df_lagged.columns:
    df_lagged[f'{col}_lag1'] = df_lagged[col].shift(1)

# Drop the first year (no lag available for 2002)
df_lagged = df_lagged.dropna()

# Add 'year' as a numeric feature (needed for modeling)
df_lagged['year'] = df_lagged.index

# Define the feature columns: lag values + year
feature_cols = [f'{col}_lag1' for col in df_pivot.columns] + ['year']

# Define future years to predict
future_years = list(range(2021, 2026))

# Create train and test sets based on year
train = df_lagged[df_lagged['year'] <= 2018]
test = df_lagged[(df_lagged['year'] > 2018) & (df_lagged['year'] <= 2020)]

# Quick check
print("Train years:", train.index.tolist())
print("Test years:", test.index.tolist())
print("Feature columns:", feature_cols)

Train years: [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
Test years: [2019, 2020]
Feature columns: ['Carbon_monoxide_lag1', 'Nitrogen_oxides_lag1', 'PM10_lag1', 'PM2.5_lag1', 'Sulphur_dioxide_lag1', 'VOCs_lag1', 'year']


In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

lr_results = {}
future_linear_predictions = pd.DataFrame(index=future_years)

for pollutant in df_pivot.columns:
    X_train = train[feature_cols].values
    y_train = train[pollutant].values

    X_test = test[feature_cols].values
    y_test = test[pollutant].values

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred_test = model.predict(X_test)

    # Save evaluation
    r2 = r2_score(y_test, y_pred_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

    lr_results[pollutant] = {
        'r2_score': round(r2, 4),
        'rmse': round(rmse, 4)
    }

# Show updated results
pd.DataFrame(lr_results).T

,r2_score,rmse
Carbon_monoxide,-2.593782e+06,617.4000
Nitrogen_oxides,-5.699738e+04,1053.0977
PM10,-1.532947e+03,217.2123
PM2.5,-8.900747e+02,155.7927
Sulphur_dioxide,-1.857801e+04,531.3020
VOCs,-3.732238e+02,945.6973


In [16]:
# from sklearn.ensemble import RandomForestRegressor

# # Initialize results
# rf_results = {}
# rf_future_predictions = pd.DataFrame(index=future_years)

# # Use year as the single input feature
# X_train = train.index.values.reshape(-1, 1)
# X_test = test.index.values.reshape(-1, 1)
# X_future = np.array(future_years).reshape(-1, 1)

# for pollutant in train.columns:
#     y_train = train[pollutant].values
#     y_test = test[pollutant].values

#     model = RandomForestRegressor(n_estimators=100, random_state=42)
#     model.fit(X_train, y_train)

#     y_pred_test = model.predict(X_test)
#     y_pred_future = model.predict(X_future)

#     rf_future_predictions[pollutant] = y_pred_future

#     r2 = r2_score(y_test, y_pred_test)
#     rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

#     rf_results[pollutant] = {
#         'r2_score': round(r2, 4),
#         'rmse': round(rmse, 4)
#     }

# # View performance comparison
# pd.DataFrame(rf_results).T


In [17]:
# from xgboost import XGBRegressor

# # Initialize structures
# xgb_results = {}
# xgb_future_predictions = pd.DataFrame(index=future_years)

# # Input features
# X_train = train.index.values.reshape(-1, 1)
# X_test = test.index.values.reshape(-1, 1)
# X_future = np.array(future_years).reshape(-1, 1)

# for pollutant in train.columns:
#     y_train = train[pollutant].values
#     y_test = test[pollutant].values

#     model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
#     model.fit(X_train, y_train)

#     y_pred_test = model.predict(X_test)
#     y_pred_future = model.predict(X_future)

#     xgb_future_predictions[pollutant] = y_pred_future

#     r2 = r2_score(y_test, y_pred_test)
#     rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

#     xgb_results[pollutant] = {
#         'r2_score': round(r2, 4),
#         'rmse': round(rmse, 4)
#     }

# # View model evaluation summary
# pd.DataFrame(xgb_results).T